In [1]:
import utils
import numpy as np
import torch
from torch.autograd import Variable
import math

In [96]:
range_boundaries = Variable(torch.Tensor([-4,-2,0,2,4,100]))

In [141]:
def logpost(w,sig,x,ratings): # w is Embedding, sig is Variable, x is input data in tensor form, ratings is just ratings                 
    h = torch.sum(w(x),1).squeeze()
    
    lefts = range_boundaries[ratings]
    rights = range_boundaries[ratings+1]
    
    normlogcdflik = utils.log_difference((rights-h)/(sig),(lefts-h)/(sig))
    
    normlogcdfprior = -0.5*torch.sum(w(usefulvec)*w(usefulvec))
    
    return torch.sum(normlogcdfprior+normlogcdflik)

def rangeval(val):
    if val < range_boundaries.data.numpy()[1]:
        return 0
    elif val < range_boundaries.data.numpy()[2]:
        return 1
    elif val < range_boundaries.data.numpy()[3]:
        return 2
    elif val < range_boundaries.data.numpy()[4]:
        return 3
    else:
        return 4
    
def predict(itergroup,w):
    rmses = 0
    cntr = 0
    batchcntr = 0
    
    for batch in itergroup:
        text = batch.text[0]
        ratings = batch.ratings-1
        
        h = torch.sum(w(text),1).squeeze()
        
#         hlist = h.data.numpy().to
        
        vecfunc = np.vectorize(rangeval)
#         rmsevec = np.apply_along_axis(rangeval,0,h.data.numpy())
        rmsevec = vecfunc(h.data.numpy())
        rmsevec = np.square(rmsevec - ratings.data.numpy())
        rmses += np.sum(rmsevec)
        cntr += len(rmsevec)
        
        batchcntr += 1
    
    rmses = rmses/(cntr+0.0)
    rmses = np.sqrt(rmses)
    
    return rmses

In [142]:
BS = 1000
num_epochs = 1
eta = 0.001

In [7]:
train_iter, val_iter, test_iter, text_field = utils.load_jester(batch_size=BS, subsample_rate=1.0, load_text=True)

Loading Data, this might take several minutes
0 lines read
100000 lines read
200000 lines read
300000 lines read
400000 lines read
500000 lines read
600000 lines read
700000 lines read
800000 lines read
900000 lines read
1000000 lines read
1100000 lines read
1200000 lines read
1300000 lines read
1400000 lines read
1500000 lines read
1600000 lines read
1700000 lines read
Data Loaded


In [152]:
V = len(text_field.vocab)
usefulvec = Variable(torch.LongTensor([x for x in range(0,V)]))
batchcntr = 0
w = torch.nn.Embedding(V,1,padding_idx=text_field.vocab.stoi['<pad>'])
sig = Variable(torch.Tensor([1]),requires_grad = True)
optimizer = torch.optim.SGD([w.weight,sig],lr = eta)

for epoch in range(num_epochs):
    train_iter.init_epoch()
    for batch in train_iter:
        text = batch.text[0]
        ratings = batch.ratings-1
        users = batch.users-1
        jokes = batch.jokes-1
        
        optimizer.zero_grad()
        
        loss = logpost(w,sig,text,ratings)
        
        loss.backward()
            
        optimizer.step()
        
        batchcntr += 1
#         print batchcntr,"batch completed!"
        
        if batchcntr == 10:
            break

In [ ]:

#     lefts = [-20]*len(val_range)
#     rights = [-20]*len(val_range) 
#     for i in range(0,len(val_range)):
#         lefts[i] = range_boundaries[ratings[i]]
#         rights[i] = range_boundaries[ratings[i]+1]      
#     normlogcdflik = utils.log_difference((Variable(torch.Tensor(rights))-h)/(sig),(Variable(torch.Tensor(lefts))-h)/(sig))
    

In [136]:
def logposte(w,sig,x,ratings): # w is Embedding, sig is Variable, x is input data in tensor form, ratings is just ratings                 
    h = torch.sum(w(x),1) #.squeeze()
    
    normlogcdflik = (-0.5/(sig**2))*torch.sum((ratings.type(torch.FloatTensor)-h)**2) 
    
    normlogcdfprior = -0.5*torch.sum(w(usefulvec)*w(usefulvec))
    
    return torch.prod(torch.sum(normlogcdfprior+normlogcdflik), -1)

In [155]:
V = len(text_field.vocab)
usefulvec = Variable(torch.LongTensor([x for x in range(0,V)]))
batchcntr = 0
w = torch.nn.Embedding(V,1,padding_idx=text_field.vocab.stoi['<pad>'])
sig = Variable(torch.Tensor([1]),requires_grad = True)
optimizer = torch.optim.SGD([w.weight,sig],lr = eta)

for epoch in range(num_epochs):
    train_iter.init_epoch()
    for batch in train_iter:
        text = batch.text[0]
        ratings = batch.ratings-1
        users = batch.users-1
        jokes = batch.jokes-1
        
        optimizer.zero_grad()
        
        loss = logposte(w,sig,text,ratings)
        loss.backward()
            
        optimizer.step()
        
        batchcntr += 1
        
        if batchcntr == 2:
            break